# Kabuki 1 & Kabuki 2 Syndrome 

##### **Kabuki 1 Syndrome** - [OMIM:147920](https://omim.org/entry/147920), MONDO:0007843
- LYSINE-SPECIFIC METHYLTRANSFERASE 2D; **KMT2D** ([OMIM:602113](https://omim.org/entry/602113))

##### **Kabuki 2 Syndrome** - [OMIM:300867](https://www.omim.org/entry/300867), MONDO:0010465
- LYSINE DEMETHYLASE 6A; **KDM6A** ([OMIM:300128](https://www.omim.org/entry/300128))

#### **Genotype Phenotype Correlations**

##### KMT2D & KDM6A

- [Banka et al (2012)](https://pubmed.ncbi.nlm.nih.gov/22126750/): 
    - analyzed the KMT2D gene in a cohort of 116 patients with Kabuki syndrome, including 18 patients previously reported by Hannibal et al. (2011), and identified KMT2D variants in 74 (63.8%).
    - KABUK1 patients were more likely to have feeding problems, kidney anomalies, early breast bud development, joint dislocations, and palatal malformations in comparison with KMT2D mutation-negative patients
    - **Systematic Kabuki syndrome facial morphology study suggested that nearly all patients with typical Kabuki syndrome facial features have pathogenic KMT2D mutations, although the disorder can be phenotypically variable.**

- [Miyake et al (2013)](https://pubmed.ncbi.nlm.nih.gov/23913813/)
    - screened 81 patients with Kabuki syndrome for mutations in the MLL2 and KDM6A genes and identified MLL2 mutations in 50 (61.7%) and KDM6A mutations in 5 (6.2%)
    - Patients with **KMT2D truncating mutations (70%) had facies that were more typical of those seen in the patients originally reported with Kabuki syndrome.** High-arched eyebrows, short fifth fingers, and infantile hypotonia were more commonly seen in patients with MLL2 mutations than in those with KDM6A mutations.
    - **Only half of the patients with KMT2D mutations had short stature and postnatal growth retardation**, compared to all of the patients with KDM6A mutations.



In [1]:
import gpsea
import hpotk

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo()
print(f'Loaded HPO v{hpo.version}')
print(f"Using gpsea version {gpsea.__version__}")

Loaded HPO v2024-12-12
Using gpsea version 0.9.1


In [2]:
kabuk1_cohort = 'KMT2D'  # KABUK1
mane_tx_id_1 = 'NM_003482.4'
mane_px_id_1 = 'NP_003473.3' 

kabuk2_cohort = 'KDM6A' #KABUK2
mane_tx_id_2 = 'NM_001291415.2'
mane_px_id_2 = 'NP_001278344.1' 

In [3]:
from ppktstore.registry import configure_phenopacket_registry
phenopacket_registry = configure_phenopacket_registry()
with phenopacket_registry.open_phenopacket_store('0.1.22') as ps:
    kabuk1_phenopackets = tuple(ps.iter_cohort_phenopackets(kabuk1_cohort))
print(f"Extracted {len(kabuk1_phenopackets)} phenopackets for Kabuki syndrome 1")

Extracted 65 phenopackets for Kabuki syndrome 1


In [4]:
from ppktstore.registry import configure_phenopacket_registry
phenopacket_registry = configure_phenopacket_registry()
with phenopacket_registry.open_phenopacket_store('0.1.20') as ps:
    kabuk2_phenopackets = tuple(ps.iter_cohort_phenopackets(kabuk2_cohort))
print(f"Extracted {len(kabuk2_phenopackets)} phenopackets for Kabuki syndrome 2")
    

Extracted 81 phenopackets for Kabuki syndrome 2


## Combine cohorts
Here, we combine the phenopackets for Kabuki syndrome types 1 and 2.

In [5]:
from gpsea.preprocessing import configure_caching_cohort_creator, load_phenopackets

kabuki_phenopackets = list()
kabuki_phenopackets.extend(kabuk1_phenopackets)
kabuki_phenopackets.extend(kabuk2_phenopackets)

cohort_creator = configure_caching_cohort_creator(hpo)
kabuki_1_and_2_cohort, validation = load_phenopackets(
    phenopackets=kabuki_phenopackets, 
    cohort_creator=cohort_creator,
)

validation.summarize()

Individuals Processed: 100%|██████████| 146/146 [00:12<00:00, 11.83individuals/s]
Validated under permissive policy
Showing errors and warnings
Phenopackets
  patient #23
    individual
     warnings:
     ·`time_element` is in currently unsupported format `ontology_class`
  patient #46
    phenotype-features
      #3
       warnings:
       ·HP:0000368 is an obsolete identifier for Posteriorly rotated ears. Replace HP:0000368 with the primary term ID HP:0000358


# Genotype-Phenotype Correlation (GPC) Analysis

This notebook compares the phenotypic features of Robinow syndrome types 1 and 2.

In [8]:
from gpsea.analysis.pcats import configure_hpo_term_analysis
from gpsea.analysis.clf import prepare_classifiers_for_terms_of_interest

analysis = configure_hpo_term_analysis(hpo)

pheno_clfs = prepare_classifiers_for_terms_of_interest(
    cohort=kabuki_1_and_2_cohort,
    hpo=hpo,
)

In [11]:
from gpsea.analysis.clf import diagnosis_classifier
from gpsea.view import MtcStatsViewer

kabuki_predicates = prepare_classifiers_for_terms_of_interest(
    cohort=kabuki_1_and_2_cohort,
    hpo=hpo,
)
len(f"Total of {len(kabuki_predicates)} Kabuki syndrome predicates")
kabuki_1_disease_id = "OMIM:147920"
kabuki_2_disease_id = "OMIM:300867"



kabuki_disease_clf = diagnosis_classifier(
    diagnoses=(kabuki_1_disease_id, kabuki_2_disease_id),
    labels=('KABUK1', 'KABUK2'),
)
result = analysis.compare_genotype_vs_phenotypes(
    cohort=kabuki_1_and_2_cohort,
    gt_clf=kabuki_disease_clf,
    pheno_clfs=kabuki_predicates,
)

viewer = MtcStatsViewer()
viewer.process(result)

Code,Reason,Count
HMF01,Skipping term with maximum frequency that was less than threshold 0.4,60
HMF03,Skipping term because of a child term with the same individual counts,4
HMF05,Skipping term because one genotype had zero observations,1
HMF08,Skipping general term,120
HMF09,Skipping term with maximum annotation frequency that was less than threshold 0.4,391


In [12]:
from gpsea.view import summarize_hpo_analysis

summarize_hpo_analysis(hpo=hpo, result=result)

Diagnosis                                          OMIM:147920          \
                                                         Count Percent   
Feeding difficulties [HP:0011968]                         8/25     32%   
Motor delay [HP:0001270]                                  4/10     40%   
Recurrent infections [HP:0002719]                        30/41     73%   
Neonatal hypoglycemia [HP:0001998]                         0/5      0%   
Atrial septal defect [HP:0001631]                        11/20     55%   
Hypotonia [HP:0001252]                                   12/21     57%   
Short stature [HP:0004322]                               22/31     71%   
Hearing impairment [HP:0000365]                          22/40     55%   
Strabismus [HP:0000486]                                   6/24     25%   
Microcephaly [HP:0000252]                                 7/18     39%   
Delayed speech and language development [HP:000...       10/12     83%   
Abnormal oral cavity morphology [HP:0000163]             29/29    100%   
Abnormal palate morphology [HP:0000174]                  22/22    100%   
Prominent fingertip pads [HP:0001212]                    30/38     79%   
Prominent digit pad [HP:0011298]                         30/30    100%   
Regional abnormality of skin [HP:0011356]                30/30    100%   
Abnormal fingertip morphology [HP:0001211]               30/30    100%   
Abnormal finger morphology [HP:0001167]                  35/35    100%   
Abnormal digit morphology [HP:0011297]                   36/36    100%   
Abnormal appendicular skeleton morphology [HP:0...       38/38    100%   
Intellectual disability [HP:0001249]                     25/27     93%   
Neurodevelopmental abnormality [HP:0012759]              44/44    100%   
Neurodevelopmental delay [HP:0012758]                    39/39    100%   
Abnormal muscle tone [HP:0003808]                        12/12    100%   
Language impairment [HP:0002463]                         10/10    100%   
Abnormal communication [HP:0034434]                      10/10    100%   
Abnormal speech pattern [HP:0002167]                     10/10    100%   
Abdominal symptom [HP:0011458]                             8/8    100%   

Diagnosis                                          OMIM:300867          \
                                                         Count Percent   
Feeding difficulties [HP:0011968]                        55/63     87%   
Motor delay [HP:0001270]                                 58/61     95%   
Recurrent infections [HP:0002719]                        22/22    100%   
Neonatal hypoglycemia [HP:0001998]                       31/55     56%   
Atrial septal defect [HP:0001631]                        16/61     26%   
Hypotonia [HP:0001252]                                   52/64     81%   
Short stature [HP:0004322]                               22/46     48%   
Hearing impairment [HP:0000365]                           8/26     31%   
Strabismus [HP:0000486]                                  16/35     46%   
Microcephaly [HP:0000252]                                24/43     56%   
Delayed speech and language development [HP:000...       54/59     92%   
Abnormal oral cavity morphology [HP:0000163]             40/40    100%   
Abnormal palate morphology [HP:0000174]                  40/40    100%   
Prominent fingertip pads [HP:0001212]                    35/45     78%   
Prominent digit pad [HP:0011298]                         35/35    100%   
Regional abnormality of skin [HP:0011356]                36/36    100%   
Abnormal fingertip morphology [HP:0001211]               35/35    100%   
Abnormal finger morphology [HP:0001167]                  37/37    100%   
Abnormal digit morphology [HP:0011297]                   44/44    100%   
Abnormal appendicular skeleton morphology [HP:0...       44/44    100%   
Intellectual disability [HP:0001249]                     53/57     93%   
Neurodevelopmental abnormality [HP:0012759]              66/66    100%   
Neu